In [1]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D

from keras.layers.convolutional import MaxPooling2D, MaxPooling3D
from keras.layers import AveragePooling2D
from keras.utils import np_utils
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.utils.np_utils import to_categorical

K.set_image_dim_ordering('tf')
seed = 7
numpy.random.seed(seed)

Using TensorFlow backend.


In [17]:
def net_builder(num_classes,input_shape):
    """
    Returns convolutional net.
    """
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(input_shape, input_shape,3), activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(AveragePooling2D(pool_size=(2,2),strides=1))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    epochs = 25
    lrate = 0.01
    decay = lrate/epochs
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

def simple_net(num_classes,input_shape):
    model = Sequential()
    model.add(Conv2D(32, 3, 3, activation='tanh', input_shape=(input_shape,input_shape,3)))
    model.add(Conv2D(32, 3, 3, activation='tanh'))
    model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='tanh'))
#     model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [18]:
img_width, img_height = 64, 64

train_data_dir = '/home/bohdan/xray_project/data/train'
validation_data_dir = '/home/bohdan/xray_project/data/val'
nb_train_samples = 1000
nb_validation_samples = 500
epochs = 5
batch_size = 100

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [19]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     shear_range=0.2,
#     zoom_range=0.2,
# horizontal_flip=True
)

In [20]:
test_datagen = ImageDataGenerator(
#     rescale=1. / 255
)

In [21]:
train_generator = train_datagen.flow_from_directory(
                                                    train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary'
)

Found 4000 images belonging to 2 classes.


In [22]:
validation_generator = test_datagen.flow_from_directory(
                                                        validation_data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='binary')

Found 999 images belonging to 2 classes.


In [28]:
import pickle
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, Conv1D
from keras.layers import Convolution2D,MaxPooling2D
from keras.optimizers import SGD, Adam,Adagrad,RMSprop
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
seed = 7
np.random.seed(seed)

In [86]:
def LeNet5(img_width, img_height, img_depth):
    """
    Returns compiled LeNet5 model.
    
    Args:
         - img_width: width of the image;
         - img_height: height of the image;
         - img_depth: number of channels of the image.
         
    Returns: 
         - compiled model.
         
    """
    model = Sequential()
    model.add(Conv2D(16,(5,5), input_shape = (img_height,img_width,img_depth), activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Conv2D(32,(5,5), activation='relu')) # used tanh activation as in the original
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Conv2D(300,(5,5), activation='relu'))

    model.add(Flatten())
    model.add(Dense(300, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))

#     opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) #0.8320

    model.compile(loss="binary_crossentropy", optimizer='adam',metrics=['accuracy']) # used binary_crossentropy as
                                                                                # we have only 2 classes not 10.
    return model

In [82]:
def create_baseline(img_height,img_width,img_depth):
    # create model
    model = Sequential()
    model.add(Conv2D(6,(5,5), input_shape = (img_height,img_width,img_depth), activation='relu'))

#     model.add(Dense(100, kernel_initializer='normal', activation='relu'))
    
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [87]:
model = LeNet5(64,64,3)

In [89]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
validation_steps=nb_validation_samples // batch_size)

Epoch 1/100
10/10 [==============================] - 16s - loss: 0.6765 - acc: 0.5970 - val_loss: 7.4121 - val_acc: 0.5351
Epoch 2/100
10/10 [==============================] - 15s - loss: 0.7012 - acc: 0.5360 - val_loss: 6.4571 - val_acc: 0.5500
Epoch 3/100
10/10 [==============================] - 15s - loss: 0.6910 - acc: 0.5290 - val_loss: 7.3801 - val_acc: 0.5371
Epoch 4/100
10/10 [==============================] - 15s - loss: 0.6855 - acc: 0.5560 - val_loss: 7.3016 - val_acc: 0.5420
Epoch 5/100
10/10 [==============================] - 15s - loss: 0.6884 - acc: 0.5330 - val_loss: 5.8586 - val_acc: 0.5411
Epoch 6/100
10/10 [==============================] - 15s - loss: 0.6768 - acc: 0.5440 - val_loss: 6.9900 - val_acc: 0.5480
Epoch 7/100
10/10 [==============================] - 15s - loss: 0.6699 - acc: 0.5880 - val_loss: 5.5928 - val_acc: 0.6052
Epoch 8/100
10/10 [==============================] - 15s - loss: 0.6606 - acc: 0.6290 - val_loss: 5.8437 - val_acc: 0.6240
Epoch 9/100
10/1

10/10 [==============================] - 15s - loss: 0.6086 - acc: 0.6730 - val_loss: 6.5586 - val_acc: 0.5792
Epoch 68/100
10/10 [==============================] - 15s - loss: 0.5984 - acc: 0.6680 - val_loss: 6.5566 - val_acc: 0.5720
Epoch 69/100
10/10 [==============================] - 15s - loss: 0.6093 - acc: 0.6630 - val_loss: 5.2346 - val_acc: 0.6633
Epoch 70/100
10/10 [==============================] - 15s - loss: 0.6047 - acc: 0.6670 - val_loss: 6.2274 - val_acc: 0.6000
Epoch 71/100
10/10 [==============================] - 15s - loss: 0.6123 - acc: 0.6720 - val_loss: 6.3784 - val_acc: 0.5852
Epoch 72/100
10/10 [==============================] - 15s - loss: 0.5981 - acc: 0.6800 - val_loss: 6.2640 - val_acc: 0.5940
Epoch 73/100
10/10 [==============================] - 15s - loss: 0.6073 - acc: 0.6640 - val_loss: 6.3461 - val_acc: 0.5892
Epoch 74/100
10/10 [==============================] - 15s - loss: 0.5939 - acc: 0.6810 - val_loss: 6.0224 - val_acc: 0.6020
Epoch 75/100
10/10 [=

In [ ]:
x_tr, y_tr = datagen.flow(train, train, batch_size=32)